# Crypto Currency Neural Network Model
### Since we are dealing with prices, accuracy is an important aspect that can't be compromised ; hence, Neural Network provides high accuracy. Especially in a large sequential dataset 
#### By Group2 Fintastic : 
#### Bashayer AlAhmadi _ Hissah Alowayyid _ Hadeel Albogami _ Hanan Alfouzan

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
from keras.utils.vis_utils import plot_model

In [2]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import Sequential

# Sourcing, Gathering and Exploring Data 
### Using the date as Index and only the close prices for both BTC and USD

In [3]:
start = dt.datetime(2020,1,1)
end = dt.datetime.now()
crypto_currency = 'BTC'
against_currency = 'USD'
#Bitcoin Data
data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', start, end) 
data = data[['Close']]
data = data.rename(columns = {'Close': 'Close_BTC'})
#NVDA Data
df_NVDA=  web.DataReader('NVDA', 'yahoo', start, end) 
df_NVDA = df_NVDA[['Close']]
df_NVDA = df_NVDA.rename(columns = {'Close': 'Close_NVDA'})

#Merge Table
#data = data.merge(data_local_currency, on = 'Date')
data = data.merge(df_NVDA, on = 'Date')
data
test_data=data
test_data

,Close_BTC,Close_NVDA
Date,,
2020-01-02,6985.470215,59.977501
2020-01-03,7344.884277,59.017502
2020-01-06,7769.219238,59.264999
2020-01-07,8163.692383,59.982498
2020-01-08,8079.862793,60.095001
...,...,...
2021-11-24,56280.425781,326.739990
2021-11-26,53569.765625,315.029999
2021-11-29,57806.566406,333.760010


# Building the Model

In [4]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data.values.reshape(-1,1))  

In [5]:
prediction_days = 60
future_day = 30

In [6]:
x_train, y_train = [], []

In [7]:
#Experimental purposes :
#for x in range(prediction_days, len(scaled_data)):
#    x_train.append(scaled_data[x-prediction_days:x, 0])
#    y_train.append(scaled_data[x, 0])

# predicting the 30th day after 60 days
for x in range(prediction_days, len(scaled_data)-future_day):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x+future_day, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
#create Neural Network 
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

Epoch 1/25
28/28 [==============================] - 19s 172ms/step - loss: 0.0898
Epoch 2/25
28/28 [==============================] - 6s 199ms/step - loss: 0.0787
Epoch 3/25
28/28 [==============================] - 4s 143ms/step - loss: 0.0769
Epoch 4/25
28/28 [==============================] - 3s 92ms/step - loss: 0.0758
Epoch 5/25
28/28 [==============================] - 3s 107ms/step - loss: 0.0776
Epoch 6/25
28/28 [==============================] - 6s 206ms/step - loss: 0.0775
Epoch 7/25
28/28 [==============================] - 4s 126ms/step - loss: 0.0762
Epoch 8/25
25/28 [=========================>....] - ETA: 0s - loss: 0.0781

In [ ]:
print(model.summary())

In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

#  Evaluating Model Performance 

In [ ]:
test_start = dt.datetime(2020,1,1)
test_end = dt.datetime.now()
#test_data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', test_start, test_end)
#test_data= Above defin
actual_prices = test_data['Close_BTC'].values
total_dataset = pd.concat((data['Close_BTC'], test_data['Close_BTC']), axis=0)
model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.fit_transform(model_inputs)
x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
prediction_prices = model.predict(x_test)
prediction_prices = scaler.inverse_transform(prediction_prices)

fig, ax1 = plt.subplots()

ax1.plot(prediction_prices, color='blue', label='Predicted Prices')
ax2 = ax1.twinx() 
ax2.plot(df_NVDA['Close_NVDA'].values, color='green', label='Prices')
ax3 = ax2.twinx()
#ax3.plot(actual_prices, color='black', label='Actual Prices')
fig.tight_layout()
plt.show()
